# Notebook Lecture 5: Transfer Functions
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of response to exponential inputs, transfer functions, conversions between state-space models and transfer functions, and connections to Laplace transforms. 

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

In previous lectures, we've seen that this course addresses the control of dynamical systems. Two key course objectives are to understand how to analyze and synthesize systems to achieve specific goals. To do this effectively, we need to understand various ways of representing and analyzing dynamical systems mathematically.
In this notebook, we delve deeper into some fundamental concepts and techniques used in control systems analysis. We'll explore system responses and different system representations.
After completing this notebook, you should be able to:

- Understand and analyze a system's response to exponential inputs.
- Interpret and work with transfer functions.
- Convert between state-space models and transfer functions, and understand the benefits of each representation.
- Appreciate the role of Laplace transforms in control systems analysis and how they relate to system behavior.

### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display

# Motivation
In Lecture 4, we observed that analyzing the time response of a Linear Time-Invariant (LTI) system can be quite challenging. The response is given by:

$$y(t) = Ce^{At}x(0) + C\int_{0}^{t} e^{A(t-\tau)}Bu(\tau)d\tau + Du(t)$$

This formula provides a complete characterization of the output response of an LTI system. However, the convolution integral makes it difficult to analyze the system's behavior.

This raises the question: how can we effectively analyze the response of a system to more complex inputs?

To address this challenge, we will study the response of LTI systems to elementary inputs such as a step and an impulse. These inputs are particularly easy to analyze because the input has the same form as the output and are very rich, meaning they can be used to describe any other input through linear combinations.


# 1. Exponential Inputs

In this section, we will study the response of an LTI system to an exponential input. The exponential input is a particularly interesting input because it is an eigenfunction of the system. This means that the output of the system will be a scaled version of the input.